# Old but gold

I though this was more a reverse engineering challenge than a crypto one. In any case, the first step was to try to understand what the binary was doing at all. I am not familiar with go, but had good success using [AlphaGolang](https://github.com/SentineLabs/AlphaGolang) with IDA Pro to reasonably figure out what the code does.

Roughly speaking, it starts a web server which decrypts a token using [AES-CFB-128](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Cipher_feedback_(CFB)) and parses it using `json.Unmarshal`. The token we are given corresponds to `{"id":"3e2bf849-9748-4046-b5c7-626e0c25846c","mail":"balsn7122@balsn.tw"}` (up to json equivalence) while the token we want to give to obtain the flag is `{"id":"aaaaaaaa-bbbb-cccc-dddd-ffffffffffff","mail":"admin","type":"admin"}`.

I say up to json equivalence, because they could have provided `id` and `mail` in a different order, or have extra whitespace, or extraneous fields etc. Fortunately the challenge author was feeling nice as it turned out that was already the correct json. How do we know this? We can easily confirm this since this gives us a known decrypted-encrypted pair of blocks, which we can use to construct a small block. In particular, we construct `{             }` with 13 spaces in between.

So what can we leak from the binary? Basically, a valid json returns a 200 and an invalid json returns a 400. Oh, also it doesn't unpad correctly, which helps greatly. It only looks at the last byte and then takes away that many bytes from the array. And it can also be greater than 16, which is amazing!

Anyway, let's just write some basic boilerplate stuff, and also double-check that we do in fact know the initial plaintext.

In [1]:
from pwn import *
from Crypto.Util.Padding import pad
from base64 import urlsafe_b64decode, urlsafe_b64encode
import random, grequests

# remote
orig_b64 = 'gXBStk4yoHkbieEBCHdLB601d9N7DbtQysvkDC_xn4_xSINrgWZIJA_BRR0bVCa9KFG99c173b8AD4fXQsNt8NgassroRWfRW6elhNdKKEE6D92jxIOlJRQ8bKumdxK1'
url_base = 'old-but-gold.balsnctf.com'

# local
#orig_b64 = 'gXBStk4yoHkbieEBCHdLB6Xc2wv2ylr2587ubNc5uHoupZZZ3j67pVaucXjvbTMX85P8hxOhR6prPFy07CGPBoK7L9NKYRaV5y5nvtD2e3PTXr_3BnvzKevB2HNuMjDx'
#url_base = 'localhost:8877'

orig_ct = urlsafe_b64decode(orig_b64)
orig_pt = pad(b'{"id":"3e2bf849-9748-4046-b5c7-626e0c25846c","mail":"balsn7122@balsn.tw"}', 16)

iv = orig_ct[:16]
iv_enc = xor(orig_ct[16:32], orig_pt[:16])

# checks multiple ciphertexts to see which ones are valid
def is_valids(bs):
    urls = (f"http://{url_base}/login/{urlsafe_b64encode(b).decode()}" for b in bs)
    rs = (grequests.get(u) for u in urls)
    codes = [x.status_code for x in grequests.map(rs)]
    assert all(code in [200, 400] for code in codes)
    return [code == 200 for code in codes]

# checks if given ciphertext returns a 200 code
def is_valid(b):
    return is_valids([b])[0]

def get_last_byte_of_encrypted(block):
    startbit = orig_ct[:16] + xor(orig_ct[16:32], orig_pt[:16], b'{}') + block + bytes(15)
    return is_valids(startbit + bytes([46 ^ i]) for i in range(256)).index(True)

def check_block_encryption(dec, enc):
    assert len(dec) == len(enc) == 16
    payload = pad(b'{             }', 16)
    assert is_valid(dec + xor(enc, payload))
    print(f'Verified the encryption of {dec.hex()} to {enc.hex()}')
    
for i in range(0, 80, 16):
    check_block_encryption(orig_ct[i:i+16], xor(orig_ct[i+16:i+32], orig_pt[i:i+16]))

Verified the encryption of 817052b64e32a0791b89e10108774b07 to d6171eb759379963aff9866a17c5a6a2
Verified the encryption of ad3577d37b0dbb50cacbe40c2ff19f8f to c87fb753ac52781039ec272878630b8b
Verified the encryption of f148836b816648240fc1451d1b5426bd to 1a67d8c5ae49e8873439e4f56ee10091
Verified the encryption of 2851bdf5cd7bddbf000f87d742c36df0 to b17690f0ca2706bd28c992b5e5786823
Verified the encryption of d81ab2cae84567d15ba7a584d74a2841 to 5b63aecdeaf7d207693b6baca17015b2


With that out of the way, the rest is just a standard AES oracle attack, depending on whether you construct valid json or not. There are various ways of doing this, and the intended solution is to spam 64k queries to construct a `{}`, but this seemed excessive.

So I came up with a different method which uses less queries. I basically construct jsons of this form:
```
<----block0----><----block1----><----block2----><----block3----><----block4----><----block5---->
0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef
                {"x":"0000000000AAAAAAAAAAAAAAAABBBBBBBBBBBBBBBBCCCCCCCCCCCCCCCC"}..............
```

* `block0` is just the IV, which we already know what it encrypts to, so we can construct block1 exactly
* `block1` is our first random factor: we just take a random 10-digit number
* `block2` is the block whose encryption we want to know
* `block3` is our second random factor: we have no control over this, so just pick anything
* `block4` is just another copy of the IV, so that we have control over the next block
* `block5` is the end, it's a quote + closing brace + unpad byte at the end

The key here is that we can randomly pick `block1` and `block3` to get completely random bytes in the `A..AB..BC..C` region. Now, it turns out that anything above 0x1F is a valid json string character in this context (is it something go-specific that it doesn't decode UTF-8?). Well, we also can't have a backslash or a quote, so that's 222 valid bytes out of 256.

This means that the probability that this is a valid json is $\left(\frac{222}{256}\right)^{48} \simeq 0.001234$, or roughly 1 in 810. Once we have this, we can shift the string back by one every time to learn what each byte is. In this way, we can fully learn the encryption of `block2`.

In [2]:
# get a (block1, block3) pair that works
def get_block13(wanted_block):
    while True:
        block13s = [(xor(iv_enc, f'{{"x":"{random.randrange(999999):010}'.encode()), random.randbytes(16)) for i in range(256)]
        block2 = wanted_block
        block5 = xor(iv_enc[:2], b'"}') + bytes(13) + xor(iv_enc[-1:], bytes([14]))
        valids = is_valids(iv + block1 + block2 + block3 + iv + block5 for block1, block3 in block13s)
        if True in valids:
            return block13s[valids.index(True)]

In [3]:
def get_encrypted_block(wanted_block):
    ct1, block3 = get_block13(wanted_block)
    iv_enc = xor(orig_ct[16:32], orig_pt[:16])
    ct2 = wanted_block
    ct4 = iv

    known = [get_last_byte_of_encrypted(wanted_block)]
    while len(known) < 16:
        ct5 = xor(iv_enc[:2], b'"}') + bytes(13) + xor(iv_enc[-1:], bytes([31 + len(known)]))
        ct3s = [block3[:15 - len(known)] + bytes([i ^ b'"'[0], known[0] ^ b'}'[0]]) + bytes(len(known)-1) for i in range(256)]
        valids = is_valids(iv + ct1 + ct2 + ct3 + ct4 + ct5 for ct3 in ct3s)
        known.insert(0, valids.index(True))
    result = bytes(known)
    print(f'{wanted_block.hex()} encrypts to {result.hex()}')
    return result

# assert get_encrypted_block(iv) == iv_enc

And that's it basically. Since we can use the oracle to encrypt any arbitrary block, this means that we can just create the ciphertext normally from the plaintext.

In [4]:
#newmsg = pad(b'{"id":"3e2bf849-9748-4046-b5c7-626e0c25846c","mail":"balsn7122@balsn.tw"}',16)
newmsg = pad(b'{"id":"aaaaaaaa-bbbb-cccc-dddd-ffffffffffff","mail":"admin","type":"admin"}',16)
assert len(newmsg) == 80
m = [newmsg[i*16:i*16+16] for i in range(5)]

c1 = xor(iv_enc, m[0])
c2 = xor(get_encrypted_block(c1), m[1])
c3 = xor(get_encrypted_block(c2), m[2])
c4 = xor(get_encrypted_block(c3), m[3])
c5 = xor(get_encrypted_block(c4), m[4])

ad3577d37b0dbb02ce98e70b76a4c78f encrypts to 2fb8aaaeecf2784f08a7a502451e6a9e
4ddac8ccc1911b2c6b8ac166217a47f8 encrypts to 0fa7144332d646bd71b006d1763fa5a2
69c1722554b020db17d660f35a1dc8c3 encrypts to 6669e8f87010c3dfebbfd291aaae8751
0f05cac25271a7b282d1f0bd88dafe21 encrypts to 4e8b399bc8dd045be0dc57613a34cf57


In [5]:
url = f"http://{url_base}/login/{urlsafe_b64encode(iv+c1+c2+c3+c4+c5).decode()}"
print(grequests.map([grequests.get(url)])[0].content)

b'Hi admin BALSN{P4dd1ng_0racl3_got_Old?How_4b0ut_JS0N_ORACLE}\n'
